# **1. Tổng quan**

Ở bài tập này, dữ liệu được dùng là dữ liệu từ WorldBank Open Data. Dữ liệu chứa thông tin hơn 200 quốc gia, về các mặt như đời sống, kinh tế-xã hội, giáo dục, môi trường,.... do Ngân hàng thế giới cập nhật từ nhiều nguồn(các quốc gia, tổ chức quốc tế,..).

# **2. Thu thập dữ liệu** 

Ở bước thu thập dữ liệu, chúng em sẽ thu thập dữ liệu qua API với các tham số như đặc trưng, quốc gia, năm.

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
INDICATOR_CODES = ['SP.POP.TOTL', # Tổng dân số
                   'SP.POP.TOTL.FE.IN', # Dân số nữ
                   'SP.POP.TOTL.MA.IN', # Dân số nam
                   'SP.DYN.CBRT.IN', # Tỉ lệ sinh(Trên 1000 dân)
                   'SP.DYN.CDRT.IN', # Tỉ lệ tử(Trên 1000 dân)
                   'NY.GDP.MKTP.CD', # GDP(USD)
                   'SL.UEM.TOTL.ZS', # Tỉ lệ thất nghiệp(%)
                   'SP.URB.TOTL.IN.ZS', # Tỉ lệ dân số đô thị(%)
                   'NE.EXP.GNFS.ZS', # Xuất khẩu hàng hóa và dịch vụ(%GPD)
                   'NE.IMP.GNFS.ZS', # Nhập khẩu hàng hóa và dịch vụ(%GPD)
                   'NY.GDP.MKTP.KD.ZG', #Tốc độ tăng trưởng GPD(%)
                   'SP.URB.GROW', # Tốc độ tăng trưởng dân số đô thị hằng năm(%)
                   'SP.DYN.TFRT.IN', # Tổng tỉ suất sinh(Số con trung bình của 1 phụ nữ)
                   'SL.AGR.EMPL.ZS', # Tỉ lệ lao động ở ngành nông nghiệp(%)
                   'SL.IND.EMPL.ZS', # Tỉ lệ lao động ở ngành công nghiệp(%)
                   'SL.SRV.EMPL.ZS',  # Tỉ lệ lao động ở ngành dịch vụ(%)
                   'NV.AGR.TOTL.ZS', # Tỉ trọng ngành nông nghiệp(%GDP)
                   'NV.IND.TOTL.ZS', # Tỉ trọng ngành công nghiệp(%GDP)
                   'NV.SRV.TOTL.ZS', # Tỉ trọng ngành dịch vụ(%GDP)
                   'SE.XPD.TOTL.GD.ZS', # Tỉ lệ chi tiêu của chính phủ cho giáo dục, tổng số (% GDP)
                   'SE.XPD.PRIM.PC.ZS', # Tỉ lệ chi tiêu cho giáo dục bậc tiểu học, tổng số (% GDP)
                   'SE.XPD.SECO.PC.ZS', # Tỉ lệ chi tiêu cho giáo dục bậc trung học, tổng số (% GDP)
                   'SE.XPD.TERT.PC.ZS', # Tỉ lệ chi tiêu cho giáo dục bậc đại học, tổng số (% GDP)
                   'SL.EMP.SELF.FE.ZS', # Tỉ lệ lao động có giới tính nữ tự làm chủ (%)
                   'SL.EMP.SELF.MA.ZS', # Tỉ lệ lao động có giới tính nam tự làm chủ (%)
                   'SL.EMP.SELF.ZS', # Tỉ lệ lao động tự làm chủ (%)
                   'SH.DTH.COMM.ZS', # Tỉ lệ tử vong do bệnh truyền nhiễm (%)
                   'SH.DTH.INJR.ZS', # Tỉ lệ tử vong do chấn thương (%)
                   'SH.DTH.NCOM.ZS', # Tỉ lệ tử vong do bệnh không truyền nhiễm (%)

                   ]

In [ ]:
COUNTRY_CODES = ['AUS', 'AFG', 'BGD', 'BTN', 'BRA', 'CHL', 'CAN', 'CHN', 'CUB', 'DEU', 'DNK', 'EGY', 'ETH', 'GBR',
                  'IDN', 'IND', 'JPN', 'KHM', 'LBY', 'MMR', 'MEX', 'SGP', 'PHL', 'RUS', 'THA', 'USA', 'VNM', 'ZWE']

In [ ]:
def loadData(country_code, start, end, format = "json", per_page = 100):

    records = {}
    for indicator_code in INDICATOR_CODES:
        url = f"https://api.worldbank.org/v2/country/{country_code}/indicator/{indicator_code}?date={start}:{end}&format=json&per_page={per_page}"
        response = requests.get(url) # Lấy về kết quả sau khi gọi api

        if response.status_code == 200:

            data = response.json() # Chuyển kết quả về dạng json

            # Xử lí data để thu được các trường thông tin mong muốn
            count = data[0]['total']

            for i in range(count):
                item = data[1][i]
                feature = item['indicator']['value']
                country = item['country']['value']
                year = item['date']
                value = item['value']

                if year not in records:
                    records[year] = {"Year": year}
                
                records[year]['Country'] = country

                records[year][feature] = value
            

        df = pd.DataFrame(records.values())

    return df

In [ ]:
raw_data = pd.DataFrame()
for country in COUNTRY_CODES:
    print(country)

    df = loadData(country, 1980, 2020)
    raw_data = pd.concat([raw_data, df], ignore_index=True)

In [ ]:
raw_data.head()
raw_data.to_csv('Data.csv', index=False)

# **3. Tiền xử lý dữ liệu**

Tuy dữ liệu có từ những nguồn khá đáng tin cậy, nhưng vẫn có những dữ liệu bị thiếu(năm đó không thực hiện khảo sát,...) hoặc những dữ liệu bị chênh lệch nhiều với phần còn lại. Do đó chúng em sẽ thực hiện bước tiếp theo là phân tích, thống kê mô tả dữ liệu, xử lí các dữ liệu thiếu. Gọi chung là tiền xử lí dữ liệu

In [ ]:
import pandas as pd

In [ ]:
raw_data = pd.read_csv('Data.csv')

Đầu tiền là đánh giá sơ bộ về dữ liệu, thống kê mô tả các trường dữ liệu

In [ ]:
raw_data.shape

Dữ liệu có ... dòng và ... cột

In [ ]:
raw_data.info()

- Chỉ có cột `Country` là kiểu `object`, các cột còn lại là kiểu `numeric`, phù hợp để sau này thực hiện trực quan, đánh giá hoặc chạy các mô hình trí tuệ nhân tạo
- Các cột vẫn có giá trị rỗng

In [ ]:
# Kiểm tra số cột có giá trị rỗng
null_counts = raw_data.isnull().sum()
num_columns_with_null = (null_counts > 0).sum()
print(f"Số cột có chứa giá trị rỗng: {num_columns_with_null}")

- Vì đang có khá nhiều nước nên khi trực quan sẽ gây khó khăn cho người xem. Do đó chúng em quyết định sẽ phân loại các nước theo các nhóm `Developed`, `Developing`, `Least developed`.
- Do đó, chúng em sẽ điền các giá trị rỗng bằng giá trị trung bình của nhóm nước đó. Vì thường các nước cùng nhóm sẽ có các chỉ số gần nhau.

In [ ]:
developed_countries = ['Australia', 'Canada', 'Germany', 'Denmark', 'United Kingdom', 'Japan', 'Singapore', 'United States']
undeveloped_countries = ["Cambodia", "Myanmar", "Zimbabwe"]
# Đánh chỉ số 0 cho các nước phát triển, 1 cho các nước đang phát triển, 2 cho các nước chưa phát triển
raw_data['Development_Status'] = raw_data['Country'].apply(lambda x: 'Developed' if x in developed_countries else ('Least developed' if x in undeveloped_countries else 'Developing'))

In [ ]:
# Điền giá trị rỗng bằng trung bình của nhóm
numeric_columns = raw_data.select_dtypes(include=['number']).columns
raw_data[numeric_columns] = raw_data.groupby("Development_Status")[numeric_columns].transform(lambda x: x.fillna(x.mean()))
raw_data.to_csv("processed_data.csv", index=False)

# **4. Đặt câu hỏi**

## Hưng

**Sự khác biệt về giáo dục giữa các quốc gia như thế nào?**

Đặc trưng cần có:
- Tỷ lệ biết chữ (%)
- Tỷ lệ nhập học trung học phổ thông và đại học (%)
- Chi tiêu giáo dục (% GDP)

**Tình trạng bình đẳng giới trên thế giới như thế nào?**

Đặc trưng cần có:
- Tỷ lệ nữ tham gia vào lực lượng lao động (%)
- Sự chênh lệch thu nhập giữa nam và nữ (Gender wage gap)
- Tỷ lệ nữ trong quốc hội (%)
- Trình độ học vấn trung bình của nữ và nam

**Các yếu tố nào ảnh hưởng đến tỷ lệ tử vong trẻ sơ sinh? Tình trạng y tế, sức khỏe cộng đồng ở các nhóm nước**

Đặc trưng cần có:

- Tỷ lệ tử vong trẻ sơ sinh (Infant Mortality Rate, per 1,000 live births)
- Tỷ lệ tiêm chủng (Immunization rate, % of children vaccinated)
- Chi tiêu y tế (% GDP)
- Nguồn nước sạch (% dân số tiếp cận được)

## Khánh

## Kỳ

**Chính phủ chi bao nhiêu phần trăm GDP bình quân đầu người cho mỗi học sinh ở bậc tiểu học, trung học và đại học?**

Đặc trưng cần có:
- Tỉ lệ chi tiêu của chính phủ cho giáo dục, tổng số (% GDP)
- Tỉ lệ chi tiêu cho giáo dục bậc tiểu học, tổng số (% GDP)
- Tỉ lệ chi tiêu cho giáo dục bậc trung học, tổng số (% GDP)
- Tỉ lệ chi tiêu cho giáo dục bậc đại học, tổng số (% GDP)

**Tỷ lệ nữ giới tự làm chủ (Self-employed, female) có cao hơn hay thấp hơn so với nam giới (Self-employed, male)? Điều này phản ánh điều gì về cơ hội kinh tế giữa hai giới?**

Đặc trưng cần có:
- Tỉ lệ lao động tự làm chủ (%)
- Tỉ lệ lao động có giới tính nam tự làm chủ (%)
- Tỉ lệ lao động có giới tính nữ tự làm chủ (%)

**Tỷ lệ nữ giới tự làm chủ (Self-employed, female) có cao hơn hay thấp hơn so với nam giới (Self-employed, male)? Điều này phản ánh điều gì về cơ hội kinh tế giữa hai giới?**

Đặc trưng cần có:
- Tỉ lệ lao động tự làm chủ (%)
- Tỉ lệ lao động có giới tính nam tự làm chủ (%)
- Tỉ lệ lao động có giới tính nữ tự làm chủ (%)

**Các nguyên nhân chính gây ra sự tử vong ở các nước. Kết quả có phản ánh mức độ phát triển của hệ thống y tế không?**

Đặc trưng cần có:
- Tỉ lệ tử vong do chấn thương (%)
- Tỉ lệ tử vong do bệnh không truyền nhiễm (%)
- Tỉ lệ tử vong do bệnh truyền nhiễm (%)
